In [1]:
%load_ext autoreload
%autoreload 2
import penzai
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [3]:
# from micrlhf.gguf import GGUFReader
# import numpy as np
# phi = GGUFReader("models/phi-3-16.gguf")
# abl = GGUFReader("models/abl.gguf")
# a, (b,), c = phi["blk.0.ffn_down.weight"]
# _, (b_,), _ = abl["blk.0.ffn_down.weight"]
# diff = b.reshape(c[::-1]) - b_.reshape(c[::-1])
# u, s, vt = np.linalg.svd(diff.astype(np.float32), full_matrices=False)
# vector = u[:, 1]

In [6]:
from utils.load_sae import get_sae
import jax.numpy as jnp
sae = get_sae(20, 5)
vector = sae["W_dec"][32524]

In [4]:
from micrlhf.llama import LlamaTransformer
from transformers import AutoTokenizer


filename = "models/phi-3-16.gguf"
llama = LlamaTransformer.from_pretrained(filename, device_map="tpu:0")
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
tokenizer.padding_side = "right"

/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
from utils.activation_manipulation import replace_activation
from micrlhf.sampling import sample


prompt = "<|user|>\nWhat is the meaning of the word \"X\"?<|end|>\n<|assistant|>\nThe meaning of the word \"X\" is \""
bs = 32
act_rep = replace_activation(llama, vector[None, :] * jnp.linspace(-70, 50, bs)[:, None], prompt=prompt, tokenizer=tokenizer)
sample(act_rep, tokenizer, prompt, batch_size=bs, do_sample=True)

  0%|          | 0/64 [00:00<?, ?it/s]

['<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "used before a noun to indicate the part of a sentence that it belongs to." However, without context, the specific word intended here isn\'t clear, as it could be a placeholder for any',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "a part used for forming a sentence or expression" or more specifically, it could be interpreted as an abbreviation or placeholder for a concept, term, or expression that has not been fully specified or is',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "used as a general term to describe the part of the line that is to the left of the main point of reference." In mathematical contexts, it refers to the elements of a sequence or function that',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "included in or forming an integral part of something." In English, "post" is a shortened form of "postscript," which is a phrase or sentence added after the main body of a',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "used as a general term to refer to a collection, assemblage, or part of something" or "preposition or definite article placed before a noun to specify or clarify its meaning." However,',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "preposition typically used to indicate a connection or inclusion of something in something else, as in \'the selection was made up of many fine pieces of art\'. However, without specific context, it\'s',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "brought forward" or "used as a count word in the context of collections or assemblies. However, without additional context, it\'s hard to determine the exact intended meaning as "up"',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "preposition" or "principal part." In different contexts, it can have varying interpretations:\n\n1. As a noun, "the" is commonly used as a collective',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "prepared to be used, particularly in a list, collection, or series." The term "pu" in the word "pu-tent" is a truncation of the word "put,"',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "consisting of," "making up," or simply referring to "the beginning" or "the front part" of something. In language, "p" is the abbreviation for "point," so',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "the first part of a recording, film, or broadcast." In a more general sense, it refers to a collection of previously recorded media. The word "pel" comes from the term "play"',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "that which has been placed or assembled first." In English, "plural" is used to refer to more than one of something, while "the plural" is a grammatical term',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "the first part or element of something." In English, "pu" is the abbreviation for the word "prime," so "pu" is often used in a similar context to "prime" or',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "one," and it can also refer to the first part of something, such as a video or a series. In astronomy, "Celestial object seen by" is a popular interpretation of',
 '<s><|user|> What is the meaning of the word "X"?<|end|><|assistant|> The meaning of the word "X" is "one" in 

In [5]:
from micrlhf.llama import LlamaBlock
from micrlhf.flash import flashify
from micrlhf.sampling import sample, trange, jnp, load_tokenizer, jit_wrapper
get_resids = llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(lambda i, x:
    pz.nn.Sequential([
        pz.de.TellIntermediate.from_config(tag=f"resid_pre.{i}"),
        x
    ])
)
get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: x.startswith("resid_pre"))
get_resids_call = jit_wrapper.Jitted(get_resids)

In [6]:
def rep_w_linear(mod):
    val = mod.table.embeddings.value  # vocabulary, embedding
    return pz.nn.Linear(pz.nn.Parameter(val, "input_embed"), ["vocabulary"], ["embedding"])
get_resids_one_hot = get_resids.select().at_instances_of(pz.nn.EmbeddingLookup).apply(rep_w_linear)
get_resids_one_hot_call = jit_wrapper.Jitted(get_resids_one_hot)

In [7]:
from micrlhf.sampling import sample, trange, jnp, load_tokenizer, jit_wrapper
from tqdm.auto import trange
from penzai.toolshed import sharding_util
import dataclasses
from functools import partial
import jax

@jax.jit
def loss_fn(logits, inputs):
    losses = pz.nx.nmap(lambda l, i: jnp.take_along_axis(jax.nn.log_softmax(l[:-1], -1), i[1:, None], 1)[:, 0].mean()
                        )(logits.untag("seq", "vocabulary"), inputs.tokens.untag("seq"))
    return -losses

bs_start = llama.mesh.shape["dp"]
tokens_init = tokenizer.encode("<|user|>\nX X X X X X X X X X X X X X X X X X X X<|end|>\n<|assistant|>\n")
optim_mask = [token == 1060 for token in tokens_init]
tokens_init = np.asarray(tokens_init)
MAX_ELITES = 4
tokens_init = np.repeat(tokens_init[None, :], MAX_ELITES, axis=0)
seed = 21
np.random.seed(seed)
tokens_init[:, optim_mask] = np.random.randint(100, tokenizer.vocab_size, tokens_init[:, optim_mask].shape)
def tokens_to_array(tokens):
    token_array = jnp.asarray(tokens)
    if len(token_array) >= bs_start:
        token_array = jax.device_put(token_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
    token_array = pz.nx.wrap(token_array, "batch", "seq")
    return token_array
def run_tokens(token_array, grad_metric=None):
    if not isinstance(token_array, pz.nx.NamedArray):
        token_array = tokens_to_array(token_array)
    inputs = llama.inputs.from_basic_segments(token_array)
    if grad_metric:
        @partial(jax.grad, has_aux=True)
        def lwg(x):
            logits, resids = get_resids_one_hot_call(dataclasses.replace(inputs, tokens=x))
            resids = {resid.tag: resid.value for resid in resids}
            metric = grad_metric(logits, resids, inputs)
            return metric, (logits, resids)
        vocab = llama.select().at_instances_of(pz.nn.EmbeddingLookup).get_sequence()[0].table.embeddings.value.named_shape["vocabulary"]
        one_hots = pz.nx.nmap(lambda x: jax.nn.one_hot(x, vocab))(inputs.tokens).tag("vocabulary")
        grad, (logits, resids) = lwg(one_hots)
    else:
        logits, resids = get_resids_call(inputs)
    losses = loss_fn(logits, inputs)
    if not grad_metric:
        resids = {resid.tag: resid.value for resid in resids}
    return_vals = logits, losses, resids
    if grad_metric:
        return_vals = return_vals + (grad,)
    return return_vals

mask = jax.device_put(jnp.asarray(optim_mask), jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("sp")))
# @partial(jax.jit, static_argnames=("key", "candidates", "expected_changes"))
def algo_iteration(elites, vector, key="resid_pre.16", candidates=128, seed=13, expected_changes=3, max_inv_temp=4, topk=128):
    elites = elites.untag("solutions").tag("batch")
    logits, _, _, grads = run_tokens(elites, grad_metric=lambda _l, r, _i: (r[key][{"seq": -1}].untag("embedding") * vector).sum().data_array.mean())
    grads = pz.nx.nmap(lambda x: x >= jax.lax.top_k(x, topk)[0][-1])(grads.untag("vocabulary")).tag("vocabulary")
    logits = logits.untag("batch").tag("elites")

    def temper(logits, key, elites, grads):
        key_choice, key_random = jax.random.split(key)
        index = jax.random.randint(key_choice, (), 0, len(logits) - 1)
        key_categorical, key_uniform, key_bernoulli, key_randint, key_use_grads, key_mutations = jax.random.split(key_random, 6)
        logit = logits[index]
        elite = elites[index]
        grads = grads[index]
        logit = jnp.roll(logit, 1, 0)
        logit = logit * jax.random.uniform(key_uniform, minval=0, maxval=max_inv_temp)
        use_grads = jax.random.bernoulli(key_use_grads, p=0.7).astype(jnp.int_)
        logit = jax.lax.switch(use_grads, ((lambda x: x), (lambda x: jnp.where(grads, x, -jnp.inf))), logit)
        to_change = jax.random.bernoulli(key_bernoulli, max(.5, expected_changes - 1) / mask.sum(), mask.shape)
        definite_indices = jax.random.randint(key_randint, mask.shape[:-1], 0, mask.shape[-1])
        definite_mask = jax.nn.one_hot(definite_indices, to_change.shape[-1], dtype=jnp.bool_)
        to_change = to_change | definite_mask
        return jnp.where(mask & to_change,
                         jax.random.categorical(key_categorical, logit),
                         elite)
    tempered_samples = pz.nx.nmap(temper)(
        logits.untag("elites", "seq", "vocabulary"),
        pz.nx.wrap(jax.random.split(jax.random.key(seed), candidates), "batch"),
        elites.untag("batch", "seq"),
        grads.untag("batch", "seq", "vocabulary")).tag("seq")
    tempered_samples = sharding_util.name_to_name_device_put(tempered_samples, llama.mesh, dict(batch="dp", seq="sp"))
    _, new_losses, new_resids = run_tokens(tempered_samples)
    new_scores = (new_resids[key][{"seq": -1}].untag("embedding") * vector).sum().astype(new_losses.dtype)
    metrics = pz.nx.nmap(lambda *xs: jnp.stack(xs))(new_losses, new_scores).tag("metrics")
    solution_axes = [k for k in tempered_samples.named_shape.keys() if k != "seq"]
    solutions = tempered_samples.untag(*solution_axes).flatten().tag("solutions").unwrap("solutions", "seq")
    metrics = metrics.untag(*(k for k in solution_axes if k != "seq")).flatten().tag("solutions").unwrap("solutions", "metrics")
    return solutions, metrics


best_metrics = None
best = tokens_to_array(tokens_init).untag("batch").tag("solutions")
xent_min = 10
xent_max = 100
weights = jnp.stack((
    jnp.linspace(-xent_max, -xent_min, MAX_ELITES),
    jnp.ones(MAX_ELITES),
), -1)
@partial(jax.jit, donate_argnums=(0, 1))
def combine_solutions(best_metrics, best, metrics, solutions):
    if best_metrics is not None:
        best_metrics = jnp.concatenate((best_metrics, metrics), 0)
        best = pz.nx.nmap(lambda a, b: jnp.concatenate((a, b)))(
            best.untag("solutions"),
            pz.nx.wrap(solutions, "solutions", "seq").untag("solutions")
        ).tag("solutions").unwrap("solutions", "seq")
    else:
        best_metrics = metrics
        best = solutions
    elite_mask = (best_metrics[None, :] * weights[:, None]).sum(-1).argmax(1)
    best_metrics = best_metrics[elite_mask]
    best = pz.nx.wrap(best[elite_mask], "solutions", "seq")
    return best_metrics, best
for seed in (bar := trange(1_000)):
    solutions, metrics = algo_iteration(best, vector, seed=seed)
    best_metrics, best = combine_solutions(best_metrics, best, metrics, solutions)
    m = {}
    for index in range(MAX_ELITES):
        i = index
        m |= {f"decoded.{i}": tokenizer.decode(best[{"solutions": index}].unwrap("seq")),
              f"loss.{i}": best_metrics[index][0], f"score.{i}": best_metrics[index][1]}
    bar.set_postfix(**m)

  0%|          | 0/1000 [00:00<?, ?it/s]

/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/jax/_src/numpy/array_methods.py:68: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in astype is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return lax_numpy.astype(arr, dtype, copy=copy, device=device)
/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/jax/_src/numpy/array_methods.py:68: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in astype is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return lax_numpy

ValueError: RESOURCE_EXHAUSTED: Error allocating device buffer: Attempting to allocate 375.75M. That was not possible. There are 368.09M free.; (0x0x0_HBM0)

: 

In [ ]:
_, resids = get_resids_call(inputs)